In [13]:
import pandas as pd
import os
import ast

In [14]:
df_credits = pd.read_csv('credits.csv')
df_credits_cast = df_credits[['cast']].copy()
df_credits_crew = df_credits[['crew']].copy()

In [15]:
# Desanidado de Cast,Crew y limpieza
def desanidar(df, column_name='cast'):
    def desanidar_list(list):
        
        if pd.isna(list):
            return []
        try:
            return ast.literal_eval(list)
        except (ValueError, SyntaxError):
            return []
    
    desanidado = df[column_name].apply(desanidar_list)
    desanidado_df = desanidado.explode().reset_index(drop=True)

    cast_df = pd.json_normalize(desanidado_df)
    
    return cast_df

df_cast_limpio = desanidar(df_credits_cast,column_name='cast')
df_crew_limpio = desanidar(df_credits_crew,column_name='crew')

In [16]:
df_cast_limpio = df_cast_limpio.rename(columns={
    'character': 'Voces',
    'id': 'actor_id',
    'name': 'name_actor',
})

In [17]:
def csv_carpeta(df, nombre_archivo, carpeta='Data'):

    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    ruta_archivo = os.path.join(carpeta, nombre_archivo)

    df.to_csv(ruta_archivo, index=False)

In [18]:
# Cargando Dataframes Limpios
csv_carpeta(df_cast_limpio[['cast_id','actor_id','name_actor','credit_id']],'cast_limpio.csv',carpeta='Data')
csv_carpeta(df_crew_limpio,'crew_limpio.csv',carpeta='Data')
